In [ ]:
from pint import UnitRegistry

import numpy as np
import tabulate
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [ ]:
# Инициируем работу с единицами измерения
ureg = UnitRegistry()
Q = ureg.Quantity


ureg.define("person=people")
Person = People = Q(1, "person")

In [ ]:
# Табличные величины
# Воздух
air_specific_heat = 1030.00 * ureg.joule/(ureg.kg*ureg.kelvin)
air_mole_weight = 29*ureg.gram/ureg.mole
air_mole_volume = 22.4*ureg.liter/ureg.mole
air_density = (air_mole_weight/air_mole_volume).to(ureg.kg/ureg.meter**3)

air_specific_heat_vol = air_specific_heat*air_density

# Вода
# Удельная теплоёмкость
water_specific_heat = 4180.6 * ureg.joule/(ureg.kg*ureg.kelvin)
# Удельная теплота парообразования
water_specific_heat_vpr = 2260000 * ureg.joule/ureg.kg

tabulate.tabulate(
    tablefmt='html',
    tabular_data=[
        ["Плотность воздуха", air_density],
        ["Обёмная удельная теплоёмкость воздуха", air_specific_heat_vol],
    ]
)

In [ ]:
# Таблица влажности воздуха

abs_hum_unit = ureg.gram/ureg.meter**3

air_table_temperatures = [Q(x, ureg.celsius).to(ureg.kelvin) for x in [0, 5, 10, 15, 20, 30, 35]]
air_table_abs_humidies = [x*abs_hum_unit for x in [4.6, 6.5, 9.2, 12.8, 17.5, 31.8, 42.1]]

air_table_temperatures_untyped = Q.from_list(air_table_temperatures)
air_table_abs_humidies_untyped = Q.from_list(air_table_abs_humidies)

air_table_spline = CubicSpline(air_table_temperatures_untyped, air_table_abs_humidies_untyped)

def air_max_humidity(temp: float) -> float:
    return air_table_spline(temp.to(ureg.kelvin)/ureg.kelvin)*abs_hum_unit

temps = np.linspace(min(air_table_temperatures_untyped), max(air_table_temperatures_untyped), 1000)


air_max_humidity(Q(22, ureg.celsius))
plt.plot(
    Q.from_list(air_table_temperatures_untyped, ureg.celsius),
    air_table_abs_humidies_untyped,
    'o'
)
plt.plot(
    Q.from_list(temps, ureg.celsius),
    air_table_spline(temps),
    '-',
)
plt.show()

In [ ]:
# Входные параметры:
# 1. Жилая площадь
living_area = 70*ureg.meter**2
ceiling_height = 2.6*ureg.meter

# 2. Количество людей
people_count = 3*People

# 3. Температура
street_temp = Q(-40, ureg.celsius)
internal_temp = Q(24, ureg.celsius)
supply_air_temp = Q(15, ureg.celsius)

# 4. Параметры воздухообмена
required_air_per_person = 30 * ureg.meter**3/(Person*ureg.hour)
living_space_airflow_k = 1 * 1/ureg.hour

# 5. Параметры влажности воздуха
required_humidity = 40*ureg.percent
# 6. Начальная температура воды
water_starting_temp = Q(10, ureg.celsius)


In [ ]:
living_volume = (living_area*ceiling_height).to(ureg.meter**3)
living_volume

In [ ]:
required_airflow_by_people = required_air_per_person*people_count

required_airflow_by_living_volume = living_volume*living_space_airflow_k
required_airflow = max(required_airflow_by_people, required_airflow_by_living_volume)

tabulate.tabulate(
    tablefmt='html',
    tabular_data=[
        ["Жилой объём", living_volume],
        ["Приток по людям", required_airflow_by_people],
        ["Приток по воздухообмену", required_airflow_by_living_volume],
        ["Приток", required_airflow],
    ]
)

In [ ]:
# Энергетика вентиляции
# 1. Нагрев воздуха
air_heat_power = (required_airflow*air_specific_heat_vol*(supply_air_temp-street_temp).to(ureg.kelvin)).to(ureg.kilowatt)
# 2. Увлажнение воды
evaporation_energy = (Q(100, ureg.celsius) - water_starting_temp).to(ureg.kelvin)*water_specific_heat + water_specific_heat_vpr

humidification_power = (required_airflow * air_max_humidity(internal_temp).to(ureg.kg/ureg.meter**3) * float(required_humidity)*evaporation_energy).to(ureg.kilowatt)


tabulate.tabulate(
    tablefmt='html',
    tabular_data=[
        ["Мощность на нагрев воздуха", air_heat_power],
        ["Мощность на увлажнение", humidification_power],
        ["Суммарная мощность", sum([air_heat_power, humidification_power])],
    ]
)